In [1]:
import os 
import sys

In [2]:
%pwd

'd:\\collab_projects\\South_German_Credit_Risk\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\collab_projects\\South_German_Credit_Risk'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gbiamgaurav/South_German_Credit_Risk.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gbiamgaurav" 
os.environ["MLFLOW_TRACKING_PASSWORD"]="352861024ebede76aa75cf4ea021ee1f74b0e39f"

### Create Entity

In [26]:
from dataclasses import dataclass
from pathlib import Path 


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path 
    model_path: Path 
    all_params: dict 
    metric_file_name: Path 
    target_column: str 
    mlflow_uri: str

### Create ConfigurationManager

In [27]:
from South_German_Bank.constants import * 
from South_German_Bank.utils.common import read_yaml, create_directories, save_json

In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/gbiamgaurav/South_German_Credit_Risk.mlflow"
        )

        return model_evaluation_config

### Create Components

In [29]:
import os 
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np 
import joblib
from South_German_Bank.constants import * 
from South_German_Bank.utils.common import read_yaml, create_directories, save_json

In [30]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        acc = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred)
        p_score = precision_score(actual, pred)
        r_score = recall_score(actual, pred)
        ras = roc_auc_score(actual, pred)

        return acc, f1, p_score, r_score, ras
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(self.config.target_column, axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(X_test)

            (acc, f1, p_score, r_score, ras) = self.eval_metrics(y_test, predicted_qualities)


            # Saving metrics as local

            scores = {"accuracy_score": acc, "f1_score": f1, "precision_score": p_score, "recall_score": r_score, "roc_auc_score": ras}

            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy_score", acc)
            mlflow.log_metric("f1_score", f1)
            mlflow.log_metric("precision_score", p_score)
            mlflow.log_metric("recall_score", r_score)
            mlflow.log_metric("roc_auc_score", ras)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGBClassifier")
            else:
                mlflow.sklearn.log_model(model, "model")

### Create Pipeline

In [31]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[{asctime}s: INFO: common: yaml file: config\config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/model_evaluation]
[{asctime}s: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


d:\collab_projects\South_German_Credit_Risk\credit_risk\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\collab_projects\South_German_Credit_Risk\credit_risk\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'XGBClassifier' already exists. Creating a new version of this model...
2023/07/31 17:34:49 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: XGBClassifier, version 4
Created versi